In [1]:
import torch
import pyro

In [6]:
true_coefs = torch.tensor([1., 2., 3.])
data = torch.randn(2000, 3)
dim = 3
labels = pyro.distributions.Bernoulli(logits=(true_coefs * data).sum(-1)).sample()

def model(data):
    coefs_mean = torch.zeros(dim)
    coefs = pyro.sample('beta', pyro.distributions.Normal(coefs_mean, torch.ones(3)))
    y = pyro.sample('y', pyro.distributions.Bernoulli(logits=(coefs * data).sum(-1)), obs=labels)
    return y

hmc_kernel = pyro.infer.HMC(model, step_size=0.0855, num_steps=4)
mcmc = pyro.infer.MCMC(hmc_kernel, num_samples=500, warmup_steps=100)
mcmc.run(data)
mcmc.get_samples()['beta'].mean(0)  

Sample: 100%|█| 600/600 [00:00, 754.76it/s, step size=7.89e-01, acc. prob=0.812]


tensor([0.9851, 1.8943, 2.9341])